In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,15526
2,Huelva,Confirmados PDIA 14 días,3463
3,Huelva,Tasa PDIA 14 días,"678,0318085612529"
4,Huelva,Confirmados PDIA 7 días,2210
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,54
629,Municipio de Huelva sin especificar,Total Confirmados,247
630,Municipio de Huelva sin especificar,Curados,116


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  15526.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  4295.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 275 personas en los últimos 7 días 

Un positivo PCR cada 169 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,15526.0,2210.0,3463.0,510743.0,432.702945,678.031809,202.0
Huelva-Costa,9168.0,1478.0,2276.0,288115.0,512.989605,789.962341,125.0
Condado-Campiña,4324.0,423.0,686.0,155057.0,272.802905,442.417950,47.0
Isla Cristina,1166.0,424.0,599.0,21264.0,1993.980436,2816.967645,35.0
Sierra de Huelva-Andévalo Central,1788.0,255.0,410.0,67571.0,377.380829,606.769176,33.0
Huelva (capital),4295.0,522.0,852.0,143663.0,363.350341,593.054579,30.0
Cartaya,598.0,98.0,139.0,19974.0,490.637829,695.904676,17.0
Lepe,1229.0,139.0,207.0,27431.0,506.725967,754.620685,16.0
Ayamonte,538.0,120.0,171.0,20946.0,572.901747,816.384990,15.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Encinasola,93.0,44.0,83.0,1305.0,3371.647510,6360.153257,6.0
Villalba del Alcor,277.0,92.0,167.0,3338.0,2756.141402,5002.995806,5.0
Isla Cristina,1166.0,424.0,599.0,21264.0,1993.980436,2816.967645,35.0
Calañas,118.0,53.0,71.0,2791.0,1898.960946,2543.891078,12.0
Villanueva de las Cruces,8.0,6.0,8.0,373.0,1608.579088,2144.772118,0.0
Berrocal,5.0,5.0,5.0,312.0,1602.564103,1602.564103,0.0
Cerro de Andévalo (El),93.0,26.0,34.0,2364.0,1099.830795,1438.240271,2.0
Palma del Condado (La),696.0,67.0,111.0,10761.0,622.618716,1031.502648,8.0
Villarrasa,111.0,19.0,20.0,2176.0,873.161765,919.117647,1.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Aroche,78.0,4.0,18.0,3073.0,130.165962,585.746827,0.0,0.222222
Cortegana,167.0,10.0,36.0,4666.0,214.316331,771.538791,1.0,0.277778
Bonares,94.0,7.0,22.0,6058.0,115.549686,363.156157,2.0,0.318182
Campillo (El),58.0,1.0,3.0,2023.0,49.431537,148.294612,0.0,0.333333
Lucena del Puerto,115.0,1.0,3.0,3371.0,29.664788,88.994364,0.0,0.333333
Niebla,121.0,1.0,3.0,4117.0,24.289531,72.868594,0.0,0.333333
Alosno,82.0,12.0,33.0,3923.0,305.888351,841.192965,0.0,0.363636
Rociana del Condado,149.0,8.0,21.0,7855.0,101.845958,267.345640,0.0,0.380952
Puebla de Guzmán,49.0,10.0,24.0,3073.0,325.414904,780.995770,0.0,0.416667
